In [34]:
import nbimporter
import numpy as np
import random
import time
import math

from chromosome import Chromosome

# class EvolManager

In [98]:
class EvolManager:
    def __init__(
        self,
        genes_per_chrom,
        pop,
        generations,
        reproductive_pool_size = None,
        offspring = None,
        mutation_rate = 0.2,
        reproductive_coefficient = 1 / 3,
        ):

        if reproductive_pool_size is None:
            reproductive_pool_size = math.floor(pop * 0.8)

        if offspring is None:
            offspring = math.floor(reproductive_pool_size * 1.5)

        self.__n = genes_per_chrom
        self.__population = pop
        self.__generations = generations
        self.__current_generation = 0
        self.__chrom_array = []
        self.__solutions = []
        self.__sol_generations = np.array([], dtype = int)
        self.__reproductive_pool = []
        self.__sorted_chrom_array = []
        self.__reproductive_pool_size = reproductive_pool_size
        self.__offspring = []
        self.__offspring_size = int(offspring)
        self.__big_array = []
        self.__solution_times = np.array([])
        self.__mutation_rate = mutation_rate
        self.__reproductive_coefficient = reproductive_coefficient
        self.__solution_times = np.array([], dtype = float)

        self.__starting_time = None

        self.populate()


    def populate(self):
        proto_chrom = Chromosome(list(range(self.__n)))

        for i in range(self.__population):
            self.__chrom_array.append( 
                Chromosome(proto_chrom.random_vec()) )

        for i in range(len(self.__chrom_array)):
            new_sol = True
            if self.__chrom_array[i].fitness() == 1.00:
                for j in range(len(self.__solutions)):
                    if np.array_equal( self.__chrom_array[i].get_positions(),
                        self.__solutions[j].get_positions() ):

                        new_sol = False
                        break
                if new_sol:
                    self.__solutions.append(self.__chrom_array[i])
                    self.__sol_generations = np.append(self.__sol_generations, 
                        self.__current_generation)
                    self.__solution_times = np.append(self.__solution_times, 0)
        print("It's populated over here!")


    def get_chrom_array(self):
        return self.__chrom_array


    def get_solutions(self):
        return self.__solutions


    def get_solution_times(self):
        return self.__solution_times

    
    def get_sol_generations(self):
        return self.__sol_generations


    def get_best_chrom_fitness(self):
        ordered_popul = sorted( self.__chrom_array, 
            key = lambda chrom: - chrom.fitness() )
        return ordered_popul[0].fitness()


    def get_best_chrom_mutual_threats(self):
        ordered_popul = sorted( self.__chrom_array, 
            key = lambda chrom: - chrom.fitness())
        return ordered_popul[0].mutual_threats()


    def get_reproductive_pool(self):
        return self.__reproductive_pool


    def get_offspring(self):
        return self.__offspring

    
    def get_current_generation(self):
        return self.__current_generation


    def set_chrom_array(self, vec_array):
       
        self.__n = len(vec_array[0])
        self.__population = len(vec_array)
        self.__current_generation = 0
        self.__chrom_array = []
        self.__solutions = []
        self.__sol_generations = np.array([], dtype = int)
        self.__reproductive_pool = []
        self.__sorted_chrom_array.clear()
        self.__offspring.clear()
        self.__big_array.clear()
        self.__solution_times = np.array([])

        for i in vec_array:
            self.__chrom_array.append( Chromosome(i) )


    # show_chromosomes
    def show_chromosomes_positions(self):
        print("__chrom_array members: ")
        for i in self.__chrom_array:
            print(f"{i.get_positions()}", end = " ") 
            print(f"fitness: {i.fitness()}")



    def show_solutions(self):
        print("Solutions: ")
        for i in range(len(self.__solutions)):
            print(f"solution [{i}]: {self.__solutions[i].get_positions()}.", 
                end = " ") 
            if len(self.__sol_generations) > 0:
                print(f"Generation: {self.__sol_generations[i]}", end = " ")
            # else:
            #     print()
            
            print(f"Time: {self.__solution_times[i]}")

        if len(self.__solutions) > 2:
            if ( self.__solutions[len(self.__solutions) - 1].get_positions() 
                    == self.__solutions[len(self.__solutions) 
                    - 2].get_positions() ).all():
                print("last solution repeated")


    def make_reproductive_pool(self):
        self.__reproductive_pool.clear()

        # Sort chromosomes by fitness in descending order
        self.__sorted_chrom_array = sorted(self.__chrom_array, 
                key = lambda crom: - crom.fitness())

        c = 0
        while len(self.__reproductive_pool) < self.__reproductive_pool_size:
            bool_signal = False
            d = c % len(self.__sorted_chrom_array)

            if ( len(self.__sorted_chrom_array ) <= (self.__population / 2) ):
                self.__sorted_chrom_array.append( Chromosome( self.__chrom_array[0].random_vec() ) )

            elif self.__n <= 2 or ( c > self.__population and len(self.__reproductive_pool) == 0 ):
                self.__reproductive_pool = self.__sorted_chrom_array[:self.__reproductive_pool_size]

            # Chromosomes have a probability of entering the reproductive pool proportional to their fitness,
            # unless a entire iteration was made without adding any chromosome to the reproductive pool.
            # Chromosomes with fitness = 0 usually don't enter the reproductive pool.
            elif random.random() < (self.__sorted_chrom_array[ d ].fitness() * self.__reproductive_coefficient):
                self.__reproductive_pool.append(self.__sorted_chrom_array[ d ])

            # If a solution is found, it is removed from the reproductive pool 
            # and a random chromosome is added to the population.
            if self.__sorted_chrom_array[ d ].fitness() == 1.000000:
                if len(self.__solutions) == 0:
                    self.__solutions.append(self.__sorted_chrom_array[ d ])
                    self.__sol_generations = np.append(self.__sol_generations, self.__current_generation)
                    ending = time.perf_counter()
                    lapse = ending - self.__starting_time
                    self.__solution_times= np.append(self.__solution_times, lapse)
                    print("Time for first solution:")
                    print(lapse)

                    # self.__reproductive_pool.append(self.__sorted_chrom_array[ d ].make_child())

                    self.__chrom_array.append( Chromosome( self.__chrom_array[0].random_vec() ) )
                    self.__chrom_array.remove(self.__sorted_chrom_array[ d ])
                    self.__sorted_chrom_array.pop( d )
                    
                    continue
                
                
                for i in range( len(self.__solutions) ):
                    if ( self.__solutions[i].get_positions() 
                                == self.__sorted_chrom_array[ d ].get_positions() ).all():
                        self.__chrom_array.append( Chromosome( self.__chrom_array[0].random_vec() ) )
                        # self.__reproductive_pool.append(self.__sorted_chrom_array[ d ].make_child())
                        if ( self.__sorted_chrom_array[ d ] in self.__chrom_array ):
                            self.__chrom_array.remove(self.__sorted_chrom_array[ d ])
                        
                        

                        if ( self.__sorted_chrom_array[ d ] in self.__reproductive_pool ):
                            self.__reproductive_pool.remove(self.__sorted_chrom_array[ d ])

                        self.__sorted_chrom_array.pop( d )

                        bool_signal = True

                        break
                
                if bool_signal:
                    continue

                else:
                    self.__solutions.append(self.__sorted_chrom_array[ d ])
                    self.__sol_generations = np.append(self.__sol_generations, self.__current_generation)
                    ending = time.perf_counter()
                    lapse = ending - self.__starting_time
                    self.__solution_times = np.append(self.__solution_times, lapse)
                    print(f"Time for new solution: { lapse }")

            c += 1



    def reproductive_season(self):
        self.__offspring.clear()
        for i in range (self.__offspring_size):
            self.__offspring.append( self.__reproductive_pool
                [i % len(self.__reproductive_pool) ].make_child() )

    
    def replacement(self):
        self.__big_array = self.__chrom_array + self.__offspring
        self.__big_array = sorted(self.__big_array, key = lambda crom: - crom.fitness())
        self.__chrom_array = self.__big_array[:self.__population]


    def greedy_next_generation(self):
        print(f"Current generation: {self.__current_generation}")
        
        self.make_reproductive_pool()
        self.reproductive_season()
        self.replacement()


    def greedy_evolution(self):
        self.__current_generation = 0
        self.__chrom_array.clear()
        self.__solutions.clear()
        self.__sol_generations = np.array([], dtype = int)
        self.__reproductive_pool.clear()
        self.__sorted_chrom_array.clear()
        self.__offspring.clear()
        self.__big_array.clear()
        self.__solution_times = np.array([], dtype = float)

        self.populate()
        self.__starting_time = time.perf_counter()

        while self.__current_generation < self.__generations:
            self.greedy_next_generation()
            self.__current_generation += 1

        print(f"Current generation: {self.__current_generation}")
        print(f"Population: {self.__population}")
        self.show_chromosomes_positions()
        self.show_solutions()


    def las_vegas(self, attemps):
        self.__current_generation = 0
        self.__chrom_array.clear()
        self.__solutions.clear()
        self.__sol_generations = np.array([], dtype = int)
        self.__reproductive_pool.clear()
        self.__sorted_chrom_array.clear()
        self.__offspring.clear()
        self.__big_array.clear()
        self.__solution_times = np.array([], dtype = float)
        self.__starting_time = time.perf_counter()

        proto_chrom = Chromosome(list(range(self.__n)))

        for a in range(attemps):
            chro = Chromosome(proto_chrom.random_vec())

            if chro.fitness() == 1.000000:
                new_sol = True
                for j in range(len(self.__solutions)):
                    if np.array_equal( chro.get_positions(),
                        self.__solutions[j].get_positions() ):
                        new_sol = False
                        break
                if new_sol:
                    self.__solutions.append(chro)
                    self.__solution_times = np.append(self.__solution_times, 
                        time.perf_counter() - self.__starting_time)



    def montecarlo(self, attemps):
        self.__current_generation = 0
        self.__chrom_array.clear()
        self.__solutions.clear()
        self.__sol_generations = np.array([], dtype = int)
        self.__reproductive_pool.clear()
        self.__sorted_chrom_array.clear()
        self.__offspring.clear()
        self.__big_array.clear()
        self.__solution_times = np.array([], dtype = float)
        self.__starting_time = time.perf_counter()

        proto_chrom = Chromosome(list(range(self.__n)))

        found_solutions = 0

        for a in range(attemps):
            chro = Chromosome(proto_chrom.random_vec())

            if chro.fitness() == 1.000000:
                new_sol = True
                found_solutions += 1
              
                self.__solution_times = np.append(self.__solution_times, 
                    time.perf_counter() - self.__starting_time)

        return found_solutions / attemps


# Testing

def __init__(  
> self,  
        genes_per_chrom,  
        pop,  
        generations,  
        reproductive_pool_size = None,  
        offspring = None,  
        mutation_rate = 0.2,  
        reproductive_coefficient = 1 / 3,
        ):

## _ _ init _ _ ( ), populate(), get_chrom_array()

In [5]:
evol_manager = EvolManager(5, 5, 10)
for c in evol_manager.get_chrom_array():
    print(c.get_positions())

SOLUTION FOUND: [0 2 4 1 3] SOLUTION FOUND: [1 4 2 0 3] It's populated over here!
[1 0 2 3 4]
[0 1 3 4 2]
[2 1 0 3 4]
[0 2 4 1 3]
[1 4 2 0 3]


## get_solutions()

In [6]:
evol_manager = EvolManager(5, 50, 10)
for c in evol_manager.get_chrom_array():
    print(c.get_positions())

SOLUTION FOUND: [2 4 1 3 0] SOLUTION FOUND: [2 0 3 1 4] SOLUTION FOUND: [1 3 0 2 4] It's populated over here!
[2 4 1 3 0]
[0 3 2 4 1]
[2 1 3 4 0]
[1 2 0 4 3]
[2 1 0 3 4]
[2 0 3 1 4]
[1 0 4 3 2]
[0 1 2 3 4]
[2 3 0 1 4]
[2 0 4 3 1]
[2 3 0 4 1]
[1 4 3 0 2]
[1 0 4 2 3]
[1 4 0 3 2]
[2 3 4 1 0]
[0 4 2 3 1]
[1 2 3 0 4]
[2 3 4 1 0]
[1 0 2 3 4]
[0 2 1 4 3]
[0 1 4 3 2]
[0 3 2 4 1]
[0 3 4 1 2]
[0 2 1 3 4]
[0 1 3 4 2]
[1 2 4 0 3]
[0 3 4 2 1]
[2 0 4 3 1]
[0 4 2 3 1]
[2 0 3 4 1]
[0 3 4 2 1]
[1 3 2 0 4]
[2 3 1 4 0]
[0 2 1 4 3]
[2 3 0 4 1]
[2 3 0 1 4]
[0 1 3 4 2]
[1 3 0 2 4]
[0 3 2 4 1]
[0 3 4 2 1]
[2 3 0 1 4]
[1 0 4 3 2]
[2 1 3 0 4]
[0 1 3 4 2]
[2 3 1 4 0]
[1 4 3 0 2]
[1 4 0 3 2]
[0 4 3 2 1]
[2 1 0 3 4]
[2 1 4 3 0]


In [7]:
for c in evol_manager.get_solutions():
    print(c.get_positions())

[2 4 1 3 0]
[2 0 3 1 4]
[1 3 0 2 4]


In [8]:
s0 = Chromosome([2, 0, 3, 1, 4])
s1 = Chromosome([0, 2, 4, 1, 3])
s2 = Chromosome([1, 3, 0, 2, 4])
s3 = Chromosome([1, 4, 2, 0, 3])
s4 = Chromosome([2, 4, 1, 3, 0])
s5 = Chromosome([0, 3, 1, 4, 2])

In [9]:
s0.print_board()

[[ 0 15  0  0  0]
 [ 0  0  0 15  0]
 [15  0  0  0  0]
 [ 0  0 15  0  0]
 [ 0  0  0  0 15]]


In [10]:
s1.print_board()

[[15  0  0  0  0]
 [ 0  0  0 15  0]
 [ 0 15  0  0  0]
 [ 0  0  0  0 15]
 [ 0  0 15  0  0]]


In [11]:
s2.print_board()

[[ 0  0 15  0  0]
 [15  0  0  0  0]
 [ 0  0  0 15  0]
 [ 0 15  0  0  0]
 [ 0  0  0  0 15]]


In [12]:
s3.print_board()

[[ 0  0  0 15  0]
 [15  0  0  0  0]
 [ 0  0 15  0  0]
 [ 0  0  0  0 15]
 [ 0 15  0  0  0]]


In [13]:
s4.print_board()

[[ 0  0  0  0 15]
 [ 0  0 15  0  0]
 [15  0  0  0  0]
 [ 0  0  0 15  0]
 [ 0 15  0  0  0]]


In [14]:
s5.print_board()

[[15  0  0  0  0]
 [ 0  0 15  0  0]
 [ 0  0  0  0 15]
 [ 0 15  0  0  0]
 [ 0  0  0 15  0]]


## get_solution_times()

In [15]:
evol_manager = EvolManager(5, 50, 10)
# for c in evol_manager.get_chrom_array():
#     print(c.get_positions())

SOLUTION FOUND: [0 2 4 1 3] SOLUTION FOUND: [2 0 3 1 4] SOLUTION FOUND: [1 4 2 0 3] SOLUTION FOUND: [1 4 2 0 3] SOLUTION FOUND: [0 3 1 4 2] SOLUTION FOUND: [2 4 1 3 0] It's populated over here!


In [16]:
evol_manager.get_solution_times()

array([0., 0., 0., 0., 0.])

In [17]:
evol_manager.get_sol_generations()

array([0, 0, 0, 0, 0])

## get_best_chrom_fitness(),  
## get_best_chrom_mutual_threats()

In [18]:
evol_manager = EvolManager(5, 50, 10)
# for c in evol_manager.get_chrom_array():
#     print(c.get_positions())

SOLUTION FOUND: [2 0 3 1 4] SOLUTION FOUND: [0 2 4 1 3] SOLUTION FOUND: [2 4 1 3 0] It's populated over here!


In [19]:
evol_manager.get_best_chrom_fitness()

SOLUTION FOUND: [2 0 3 1 4] SOLUTION FOUND: [0 2 4 1 3] SOLUTION FOUND: [2 4 1 3 0] SOLUTION FOUND: [2 0 3 1 4] 

1.0

In [20]:
evol_manager.get_best_chrom_mutual_threats()

SOLUTION FOUND: [2 0 3 1 4] SOLUTION FOUND: [0 2 4 1 3] SOLUTION FOUND: [2 4 1 3 0] 

0

## get_reproductive_pool()

## get_offspring()

## set_chrom_array()

In [21]:
evol_manager = EvolManager(5, 5, 10)

evol_manager.show_chromosomes_positions()


It's populated over here!
__chrom_array members: 
[1 0 2 3 4] fitness: 0.6
[2 1 4 3 0] fitness: 0.6
[1 0 3 4 2] fitness: 0.6
[0 2 3 4 1] fitness: 0.6
[1 4 3 2 0] fitness: 0.6


In [22]:
vectors_array = np.array([
    np.array([2, 0, 3, 1, 4]),
    np.array([0, 2, 4, 1, 3]),
    np.array([1, 3, 0, 2, 4]),
    np.array([1, 4, 2, 0, 3]),
    np.array([2, 4, 1, 3, 0]),
    ])

evol_manager.set_chrom_array(vectors_array)
evol_manager.show_chromosomes_positions()

__chrom_array members: 
[2 0 3 1 4] SOLUTION FOUND: [2 0 3 1 4] fitness: 1.0
[0 2 4 1 3] SOLUTION FOUND: [0 2 4 1 3] fitness: 1.0
[1 3 0 2 4] SOLUTION FOUND: [1 3 0 2 4] fitness: 1.0
[1 4 2 0 3] SOLUTION FOUND: [1 4 2 0 3] fitness: 1.0
[2 4 1 3 0] SOLUTION FOUND: [2 4 1 3 0] fitness: 1.0


## show_chromosomes_positions()

In [23]:
evol_manager = EvolManager(5, 5, 10)

It's populated over here!


In [24]:
evol_manager.show_chromosomes_positions()

__chrom_array members: 
[0 2 1 4 3] fitness: 0.6
[2 0 4 1 3] fitness: 0.8
[1 3 4 2 0] fitness: 0.8
[1 0 2 3 4] fitness: 0.6
[1 4 3 2 0] fitness: 0.6


## show_solutions()

In [25]:
evol_manager = EvolManager(5, 50, 10)

SOLUTION FOUND: [0 2 4 1 3] SOLUTION FOUND: [2 0 3 1 4] SOLUTION FOUND: [0 2 4 1 3] It's populated over here!


In [26]:
evol_manager.show_solutions()

Solutions: 
solution [0]: [0 2 4 1 3].                     Generation: 0
solution [1]: [2 0 3 1 4].                     Generation: 0


In [27]:
evol_manager.show_chromosomes_positions()

__chrom_array members: 
[2 0 3 4 1] fitness: 0.8
[1 2 4 0 3] fitness: 0.8
[0 2 4 1 3] SOLUTION FOUND: [0 2 4 1 3] fitness: 1.0
[2 3 4 0 1] fitness: 0.6
[2 1 4 0 3] fitness: 0.8
[1 0 2 3 4] fitness: 0.6
[0 3 1 2 4] fitness: 0.8
[2 4 3 1 0] fitness: 0.8
[1 2 0 3 4] fitness: 0.8
[1 2 3 0 4] fitness: 0.6
[2 0 3 1 4] SOLUTION FOUND: [2 0 3 1 4] fitness: 1.0
[2 3 4 1 0] fitness: 0.4
[2 0 4 3 1] fitness: 0.8
[1 3 4 2 0] fitness: 0.8
[0 2 4 3 1] fitness: 0.8
[0 4 3 1 2] fitness: 0.8
[1 0 3 2 4] fitness: 0.6
[0 3 4 1 2] fitness: 0.6
[1 2 4 0 3] fitness: 0.8
[2 4 3 1 0] fitness: 0.8
[1 4 3 2 0] fitness: 0.6
[0 4 2 3 1] fitness: 0.6
[0 2 1 3 4] fitness: 0.6
[0 4 3 2 1] fitness: 0.4
[1 4 0 3 2] fitness: 0.8
[2 3 4 1 0] fitness: 0.4
[1 4 0 2 3] fitness: 0.8
[0 3 2 1 4] fitness: 0.4
[1 4 3 0 2] fitness: 0.8
[2 4 1 0 3] fitness: 0.8
[0 2 3 4 1] fitness: 0.6
[1 3 2 0 4] fitness: 0.8
[1 4 3 0 2] fitness: 0.8
[1 2 0 3 4] fitness: 0.8
[0 1 2 3 4] fitness: 0.0
[1 0 2 3 4] fitness: 0.6
[0 1 4 3 2] fitness:

## make_reproductive_pool() & sorting by fitness

In [28]:
chrom_list = []
chr = Chromosome( np.arange(8) )

for i in range(5):
    chrom_list.append( Chromosome( chr.random_vec() ) )

for c in chrom_list:
    print(f"{c.get_positions()} fitness: {c.fitness()}")

[0 3 1 7 5 6 2 4] fitness: 0.9285714285714286
[1 5 7 4 6 0 3 2] fitness: 0.8214285714285714
[3 2 4 0 6 1 5 7] fitness: 0.8214285714285714
[3 2 6 0 7 5 4 1] fitness: 0.7857142857142857
[3 1 4 7 0 2 6 5] fitness: 0.9285714285714286


In [29]:
sorted_chrom_list = sorted(chrom_list, key=lambda crom: -crom.fitness())

for c in sorted_chrom_list:
    print(f"{c.get_positions()} fitness: {c.fitness()}")

[0 3 1 7 5 6 2 4] fitness: 0.9285714285714286
[3 1 4 7 0 2 6 5] fitness: 0.9285714285714286
[1 5 7 4 6 0 3 2] fitness: 0.8214285714285714
[3 2 4 0 6 1 5 7] fitness: 0.8214285714285714
[3 2 6 0 7 5 4 1] fitness: 0.7857142857142857


In [30]:
for i in chrom_list:
    print(i.get_positions())

[0 3 1 7 5 6 2 4]
[1 5 7 4 6 0 3 2]
[3 2 4 0 6 1 5 7]
[3 2 6 0 7 5 4 1]
[3 1 4 7 0 2 6 5]


## las_vegas()

In [85]:
evol_manager = EvolManager(8, 50, 10)

It's populated over here!


In [86]:
evol_manager.las_vegas(10000)


SOLUTION FOUND: [2 5 3 1 7 4 6 0] SOLUTION FOUND: [2 4 6 0 3 1 7 5] SOLUTION FOUND: [3 5 7 1 6 0 2 4] SOLUTION FOUND: [3 1 6 4 0 7 5 2] SOLUTION FOUND: [3 6 2 7 1 4 0 5] SOLUTION FOUND: [3 1 7 4 6 0 2 5] SOLUTION FOUND: [2 5 1 6 0 3 7 4] SOLUTION FOUND: [2 5 7 0 4 6 1 3] SOLUTION FOUND: [3 5 7 1 6 0 2 4] SOLUTION FOUND: [1 6 4 7 0 3 5 2] SOLUTION FOUND: [2 7 3 6 0 5 1 4] SOLUTION FOUND: [3 5 0 4 1 7 2 6] SOLUTION FOUND: [2 4 1 7 5 3 6 0] SOLUTION FOUND: [3 5 7 2 0 6 4 1] SOLUTION FOUND: [0 6 3 5 7 1 4 2] SOLUTION FOUND: [2 5 3 1 7 4 6 0] SOLUTION FOUND: [3 1 7 5 0 2 4 6] SOLUTION FOUND: [3 6 0 7 4 1 5 2] SOLUTION FOUND: [3 5 0 4 1 7 2 6] 

In [87]:
evol_manager.show_solutions()

Solutions: 
solution [0]: [2 5 3 1 7 4 6 0]. Time: 0.08795650899992324
solution [1]: [2 4 6 0 3 1 7 5]. Time: 0.10727445899829036
solution [2]: [3 5 7 1 6 0 2 4]. Time: 0.13971053600107552
solution [3]: [3 1 6 4 0 7 5 2]. Time: 0.16689469900302356
solution [4]: [3 6 2 7 1 4 0 5]. Time: 0.18766491999849677
solution [5]: [3 1 7 4 6 0 2 5]. Time: 0.24103693399956683
solution [6]: [2 5 1 6 0 3 7 4]. Time: 0.2547133210027823
solution [7]: [2 5 7 0 4 6 1 3]. Time: 0.3416507369984174
solution [8]: [1 6 4 7 0 3 5 2]. Time: 0.3665441909979563
solution [9]: [2 7 3 6 0 5 1 4]. Time: 0.47680841400142526
solution [10]: [3 5 0 4 1 7 2 6]. Time: 0.49482838500262005
solution [11]: [2 4 1 7 5 3 6 0]. Time: 0.5318971180022345
solution [12]: [3 5 7 2 0 6 4 1]. Time: 0.595322296001541
solution [13]: [0 6 3 5 7 1 4 2]. Time: 0.6780785520022619
solution [14]: [3 1 7 5 0 2 4 6]. Time: 0.6986367959980271
solution [15]: [3 6 0 7 4 1 5 2]. Time: 0.8886611850030022


# Executions

def __init__(  
> self,  
        genes_per_chrom,  
        pop,  
        generations,  
        reproductive_pool_size = None,  
        offspring = None,  
        mutation_rate = 0.2,  
        reproductive_coefficient = 1 / 3,
        ):

## greedy_evolution()


In [94]:
evol_manager = EvolManager(
    genes_per_chrom = 8, 
    pop = 1000, 
    generations = 50, 
    reproductive_pool_size = None,
    offspring = 500)

evol_manager.greedy_evolution()

SOLUTION FOUND: [3 0 4 7 1 6 2 5] SOLUTION FOUND: [3 1 6 2 5 7 0 4] SOLUTION FOUND: [3 6 4 2 0 5 7 1] SOLUTION FOUND: [2 5 1 6 4 0 7 3] It's populated over here!
SOLUTION FOUND: [3 1 7 4 6 0 2 5] It's populated over here!
Current generation: 0
SOLUTION FOUND: [3 1 7 4 6 0 2 5] SOLUTION FOUND: [3 1 7 4 6 0 2 5] SOLUTION FOUND: [3 1 7 4 6 0 2 5] SOLUTION FOUND: [3 5 7 1 6 0 2 4] SOLUTION FOUND: [2 5 1 6 4 0 7 3] Current generation: 1
SOLUTION FOUND: [3 5 7 1 6 0 2 4] SOLUTION FOUND: [2 5 1 6 4 0 7 3] SOLUTION FOUND: [3 5 7 1 6 0 2 4] SOLUTION FOUND: [3 5 7 1 6 0 2 4] Time for new solution: 0.19417622099717846
SOLUTION FOUND: [2 5 1 6 4 0 7 3] SOLUTION FOUND: [2 5 1 6 4 0 7 3] Time for new solution: 0.19435185699694557
SOLUTION FOUND: [3 5 7 1 6 0 2 4] SOLUTION FOUND: [3 5 7 1 6 0 2 4] SOLUTION FOUND: [2 5 1 6 4 0 7 3] SOLUTION FOUND: [2 5 1 6 4 0 7 3] SOLUTION FOUND: [3 0 4 7 1 6 2 5] SOLUTION FOUND: [1 4 6 3 0 7 5 2] Current generation: 2
SOLUTION FOUND: [3 0 4 7 1 6 2 5] SOLUTION FOUND

In [95]:
evol_manager.show_solutions()

Solutions: 
solution [0]: [3 1 7 4 6 0 2 5]. Generation: 0 Time: 0.0
solution [1]: [3 5 7 1 6 0 2 4]. Generation: 1 Time: 0.19417622099717846
solution [2]: [2 5 1 6 4 0 7 3]. Generation: 1 Time: 0.19435185699694557
solution [3]: [3 0 4 7 1 6 2 5]. Generation: 2 Time: 0.34999535999668296
solution [4]: [1 4 6 3 0 7 5 2]. Generation: 2 Time: 0.3503292839959613
solution [5]: [2 5 7 1 3 0 6 4]. Generation: 3 Time: 0.4952847259992268
solution [6]: [1 5 7 2 0 3 6 4]. Generation: 3 Time: 0.49545008900167886
solution [7]: [2 4 1 7 5 3 6 0]. Generation: 3 Time: 0.49562090799736325
solution [8]: [3 6 4 2 0 5 7 1]. Generation: 3 Time: 0.49584265099838376
solution [9]: [3 7 4 2 0 6 1 5]. Generation: 3 Time: 0.49618777800060343
solution [10]: [3 6 4 1 5 0 2 7]. Generation: 4 Time: 0.6266855139983818
solution [11]: [3 1 7 5 0 2 4 6]. Generation: 4 Time: 0.6268547610015958
solution [12]: [2 4 7 3 0 6 1 5]. Generation: 4 Time: 0.6272483979992103
solution [13]: [2 6 1 7 4 0 3 5]. Generation: 4 Time: 0.6

## las_vegas()

In [108]:
las_vegas_manager = EvolManager(
    genes_per_chrom = 12, 
    pop = 100, 
    generations = 100)

las_vegas_manager.las_vegas(500000)

It's populated over here!
SOLUTION FOUND: [ 3 10  6  9  1  4  8  0  2  7  5 11] SOLUTION FOUND: [ 3  5 11  9  1  6  8  2  0  7 10  4] SOLUTION FOUND: [ 2  4  7  9 11  6  1  3  0  8 10  5] SOLUTION FOUND: [ 1  6  9  0  4  7 10  3 11  8  5  2] SOLUTION FOUND: [ 5  9 11  3  8  4  2  0 10  7  1  6] SOLUTION FOUND: [ 4  9  1 10  2  6 11  0  8  3  5  7] SOLUTION FOUND: [ 5  7  0  4 11  1 10  6  3  9  2  8] SOLUTION FOUND: [ 5 10  2  9  6  4  1 11  0  7  3  8] SOLUTION FOUND: [ 4  7 11  6  1  5  8  0  3 10  2  9] SOLUTION FOUND: [ 3  5  9  1  6 11  7  2  0  8 10  4] SOLUTION FOUND: [ 3 10  6  9  2  0 11  8  5  1  4  7] SOLUTION FOUND: [ 3  7  9  4  1 10  8  5  2  0  6 11] SOLUTION FOUND: [ 5  8  1  4 10  3  0  9 11  6  2  7] SOLUTION FOUND: [ 4  6  3  9 11  1  5  2  0  7 10  8] 

In [109]:
las_vegas_manager.show_solutions()

Solutions: 
solution [0]: [ 3 10  6  9  1  4  8  0  2  7  5 11]. Time: 1.212199708999833
solution [1]: [ 3  5 11  9  1  6  8  2  0  7 10  4]. Time: 7.066774425009498
solution [2]: [ 2  4  7  9 11  6  1  3  0  8 10  5]. Time: 15.623162161005894
solution [3]: [ 1  6  9  0  4  7 10  3 11  8  5  2]. Time: 26.448378647997743
solution [4]: [ 5  9 11  3  8  4  2  0 10  7  1  6]. Time: 32.20660238500568
solution [5]: [ 4  9  1 10  2  6 11  0  8  3  5  7]. Time: 33.990454291008064
solution [6]: [ 5  7  0  4 11  1 10  6  3  9  2  8]. Time: 39.50588214100571
solution [7]: [ 5 10  2  9  6  4  1 11  0  7  3  8]. Time: 49.00669905700488
solution [8]: [ 4  7 11  6  1  5  8  0  3 10  2  9]. Time: 51.00068807401112
solution [9]: [ 3  5  9  1  6 11  7  2  0  8 10  4]. Time: 51.60318388600717
solution [10]: [ 3 10  6  9  2  0 11  8  5  1  4  7]. Time: 58.83708045800449
solution [11]: [ 3  7  9  4  1 10  8  5  2  0  6 11]. Time: 59.098508858005516
solution [12]: [ 5  8  1  4 10  3  0  9 11  6  2  7]. Time

## Montecarlo

In [110]:
montecarlo_manager = EvolManager(
    genes_per_chrom = 12, 
    pop = 100, 
    generations = 100)

ratio = montecarlo_manager.montecarlo(500000)

It's populated over here!
SOLUTION FOUND: [ 5  7  9  0  3  1 10  8  6  4  2 11] SOLUTION FOUND: [ 3  1  9  7 10  4  0  8  5 11  2  6] SOLUTION FOUND: [ 5  0  2 10  8  6  3  9 11  1  4  7] SOLUTION FOUND: [ 4  8  5  2  9  3  0  7 10  6 11  1] SOLUTION FOUND: [ 5  0  4  6  8 11  1  7  2 10  3  9] SOLUTION FOUND: [ 5  8  4  0 10  6  9 11  2  7  3  1] SOLUTION FOUND: [ 2  5  7 10  4 11  9  0  6  3  1  8] SOLUTION FOUND: [ 5  0  6  9  2  8  3  7 10  1 11  4] SOLUTION FOUND: [ 5 11  0  3  7  2  8  1  9  4  6 10] SOLUTION FOUND: [ 3  5  9  2  6 11  1  7  0 10  8  4] SOLUTION FOUND: [ 2  7 11  3  1  9  5  0  4 10  8  6] SOLUTION FOUND: [ 2  7  9  3  1  4 10  0 11  5  8  6] 

In [112]:
ratio

2.4e-05